In [ ]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.utils import np_utils
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# tf.debugging.set_log_device_placement(True)

In [ ]:
data = pd.read_json('/content/drive/My Drive/Master Thesis/Data/sample_1000000_2005_2018_cleaned.json')

In [ ]:
data.head(1)

,index,text,stars,cleaned_text,stars_reduce
1105058,2344352,Excellent food and customer service! My mom fo...,5,Excellent food and customer service ! My mom f...,4


In [ ]:
data['stars_reduce'] = data['stars']-1

In [ ]:
dummy_y = np_utils.to_categorical(data.stars_reduce)
dummy_y.shape

(1000000, 5)

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(data.cleaned_text, dummy_y, test_size=0.2, random_state = 45)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

#words_to_index is the dictionary mapping words to their respective index.
words_to_index = tokenizer.word_index

In [ ]:
maxLen = 300
vocab_len = len(words_to_index)+1
embedding_dim = 64

In [ ]:
def lstm_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = Embedding(input_dim=vocab_len, output_dim=embedding_dim, input_length=maxLen)(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.5)(X)

  X = LSTM(128)(X)

  X = Dropout(0.5)(X)

  X = Dense(5, activation='softmax')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
model = lstm_model((maxLen,))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 50)           8609550   
                                                                 
 lstm (LSTM)                 (None, 300, 128)          91648     
                                                                 
 dropout (Dropout)           (None, 300, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 5)                 645   

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

print(X_train_indices.shape)

X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

(800000, 300)


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 1e-5)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
callback = EarlyStopping(monitor='loss', patience=2)
history = model.fit(X_train_indices, Y_train, validation_data=(X_test_indices, np.asarray(Y_test)), batch_size=64, epochs=20, callbacks=[callback])

Epoch 1/20
12500/12500 [==============================] - 385s 30ms/step - loss: 1.2139 - accuracy: 0.4789
Epoch 2/20
12500/12500 [==============================] - 388s 31ms/step - loss: 1.0582 - accuracy: 0.5265
Epoch 3/20
12500/12500 [==============================] - 399s 32ms/step - loss: 1.0176 - accuracy: 0.5430
Epoch 4/20
12500/12500 [==============================] - 393s 31ms/step - loss: 0.9973 - accuracy: 0.5527
Epoch 5/20
12500/12500 [==============================] - 403s 32ms/step - loss: 0.9842 - accuracy: 0.5579
Epoch 6/20
12500/12500 [==============================] - 395s 32ms/step - loss: 0.9745 - accuracy: 0.5628
Epoch 7/20
12500/12500 [==============================] - 376s 30ms/step - loss: 0.9669 - accuracy: 0.5669
Epoch 8/20
12500/12500 [==============================] - 377s 30ms/step - loss: 0.9602 - accuracy: 0.5703
Epoch 9/20
12500/12500 [==============================] - 383s 31ms/step - loss: 0.9542 - accuracy: 0.5727
Epoch 10/20
12500/12500 [============

In [ ]:
model.evaluate(X_test_indices, Y_test)

6250/6250 [==============================] - 95s 15ms/step - loss: 0.9427 - accuracy: 0.5941


[0.9427026510238647, 0.5941299796104431]

In [ ]:
#plot accuracy
epochs = 20
plt.figure(figsize=(15, 7))
plt.plot(range(epochs), history.history['accuracy'])
plt.plot(range(epochs), history.history['val_accuracy'])
plt.legend(['training_acc', 'validation_acc'])
plt.title('Accuracy')

In [ ]:
#plot loss
plt.figure(figsize=(15, 7))
plt.plot(range(epochs), history.history['loss'])
plt.plot(range(epochs), history.history['val_loss'])
plt.legend(['training_loss', 'validation_loss'])
plt.title('Loss')

In [ ]:
preds = model.predict(X_test_indices)

In [ ]:
star_pred = np.argmax(preds, axis=1)+1
star_pred

array([5, 5, 2, ..., 5, 4, 4])

In [ ]:
model.save('/content/drive/My Drive/Master Thesis/Model/glove_lstm_2015_2018_1000000')

INFO:tensorflow:Assets written to: /content/drive/My Drive/Master Thesis/Model/glove_lstm_2015_2018_1000000/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Master Thesis/Model/glove_lstm_2015_2018_1000000/assets


In [ ]:
loaded_model = keras.models.load_model('/content/drive/My Drive/Master Thesis/Model/glove_lstm_2015_2018_1000000')

In [ ]:
# loading
with open('/content/drive/My Drive/Master Thesis/Model/keras_tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [ ]:
X_test_indices = loaded_tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
X_test_indices.shape

(200000, 300)

In [ ]:
np.testing.assert_allclose(
    model.predict(X_test_indices), loaded_model.predict(X_test_indices)
)

In [ ]:
loaded_preds = loaded_model.predict(X_test_indices)

In [ ]:
star_pred = np.argmax(loaded_preds, axis=1)+1

In [ ]:
star_pred

array([5, 5, 2, ..., 5, 4, 4])